# 02 Embeddings, Memories and Vectors

**WIP**

Literally embedding data into a model.

Historically, applications would embed data into the apps to enable search etc., so that's why it's known as embedding.

Create an embedding using a model. Pass data to OpenAI, data is returned as vectors

Vectors?

Memories?

The right approach is semantic search based on embedding vectors, which we compare against each other using cosine similarity to find a "fact" for a given specific question. See the example with a detailed description below.

An embedding is a vector of numbers that helps us understand how semantically similar or different the texts are. The closer two embeddings are to each other, the more similar their contents are

There are two things we need to understand at this point:

Why do we need to transform text into an embedding vector (i.e., numbers)? Later on, we can compare embedding vectors and figure out how similar the two texts are. We can't compare texts as such.
Why are there exactly 1536 numbers inside the embedding vector? Because the text-embedding-ada-002 model has an output dimension of 1536. It's pre-defined.

There are two things we need to understand at this point:

Why do we need to transform text into an embedding vector (i.e., numbers)? Later on, we can compare embedding vectors and figure out how similar the two texts are. We can't compare texts as such.
Why are there exactly 1536 numbers inside the embedding vector? Because the text-embedding-ada-002 model has an output dimension of 1536. It's pre-defined.

Embedding vs. Fine tuning

Step 1, 2, 3 etc.

Reference
https://stackoverflow.com/questions/74000154/openai-gpt-3-api-how-do-i-make-sure-answers-are-from-a-customized-fine-tuning/75192794#75192794